<div style='background-image: url("header.png") ; padding: 0px ; background-size: cover ; border-radius: 5px ; height: 250px'>
    <div style="float: right ; margin: 50px ; padding: 20px ; background: rgba(255 , 255 , 255 , 0.7) ; width: 50% ; height: 150px">
        <div style="position: relative ; top: 50% ; transform: translatey(-50%)">
            <div style="font-size: xx-large ; font-weight: 900 ; color: rgba(0 , 0 , 0 , 0.8) ; line-height: 100%">Tutorial by Mondaic</div>
            <div style="font-size: large ; padding-top: 20px ; color: rgba(0 , 0 , 0 , 0.5)">For Salvus version 0.11.25</div>
        </div>
    </div>
</div>

# Side-set Sources and Receivers for the Wave Equation

The previous tutorial introduced source and receivers for `SalvusCompute` where the coordinates are already known exactly. In many scenarios this is not necessarily the case. Imaging a mesh with potentially arbitrarily shaped and curved elements and you want to exactly locate a source or receiver relative to the local surface or for example at the ocean bottom. This problem is dependent on the exact mesh used for the simulation.

To this end we offer so called "side-set relative" sources and receiver.

In [ ]:
# This line helps with tab-completion of the simple_config objects.
# The IPython/Jupyter project default to a differnet inference based
# tab completion engine which unfortunately does not yet fully work
# with SalvusFlow. This is completely optional and a convenience
# option.
%config Completer.use_jedi = False

# SalvusFlow and SalvusMesh convenience interfaces.
from salvus.flow import simple_config
from salvus.mesh import simple_mesh

# We'll also use numpy.
import numpy as np


## Side-sets

side-set are just a way to flag edges (in 2D) or faces (in 3D) and giving these a name. In `SalvusCompute` these are used to specify boundary conditions, here we will use them to locate source and receiver relative to them. The following example mesh has 4 side-sets, each denoting one side of the mesh. *Please note, that the names are arbitrary and mesh dependent.*

![](./side_set_explanation.png)

## Source and receiver locations relative to a side-set

The following sketch illustrates how the location of sources and receivers on side-sets is specified.

![](./side_set_point_explanation.png)

## List of available classes

The available types of sources and receivers are exactly identical to the ones introduced in the previous tutorial, but prefixed with `SideSet` and all the coordinates are replaced with a `point`, `direction`, `side_set_name`, and `offset` parameters.

#### Receivers

* `simple_config.receiver.cartesian.SideSetPoint2D`
* `simple_config.receiver.cartesian.SideSetPoint3D`


#### Sources in acoustic media

* `simple_config.source.cartesian.SideSetScalarPoint2D`
* `simple_config.source.cartesian.SideSetScalarPoint3D`

* `simple_config.source.cartesian.SideSetScalarGradientPoint2D`
* `simple_config.source.cartesian.SideSetScalarGradientPoint3D`

#### Sources in elastic media

* `simple_config.source.cartesian.SideSetVectorPoint2D`
* `simple_config.source.cartesian.SideSetVectorPoint3D`

* `simple_config.source.cartesian.SideSetMomentTensorPoint2D`
* `simple_config.source.cartesian.SideSetMomentTensorPoint3D`

## Examples

Let us construct a tad bit of a more complicated mesh - the elements of this mesh have fourth order polynomial shapes, thus finding their surface is a non-trivial undertaking.

In [ ]:
m = simple_mesh.Cartesian2D()
m.basic.min_period_in_seconds = 1500.0
m.basic.model = "prem_iso_no_crust"
m.basic.elements_per_wavelength = 2.0
m.advanced.tensor_order = 4
mesh = m.create_mesh()

# Add a nice curve to the mesh surface.
# Note how it would not longer be trivial to add sources
# or receivers exactly at the surface of the mesh!
x_range = mesh.points[:, 0].ptp()
mesh.points[:, 1] += np.sin(mesh.points[:, 0] / x_range * 2 * np.pi) * 3e5
mesh

In [ ]:
# One receiver at the top, one at the bottom.
recs = [
    simple_config.receiver.cartesian.SideSetPoint2D(
        point=(3e6, 0.0),
        # Locate vertically.
        direction=(0.0, 1.0),
        # Top side-set.
        side_set_name="y1",
        # Followed by the normal receiver coordinates.
        station_code="A1",
        fields=["displacement"],
    ),
    # Second receiver.
    simple_config.receiver.cartesian.SideSetPoint2D(
        point=(5e6, 0.0),
        # Note that the direction vector can also be specified as the
        # name of the coordinate axis.
        direction="y",
        # Bottom
        side_set_name="y0",
        # Followed by the normal receiver coordinates.
        station_code="A2",
        fields=["displacement"],
    ),
]

srcs = simple_config.source.cartesian.SideSetVectorPoint2D(
    point=[2e6, 0.0],
    direction="y",
    side_set_name="y1",
    # Bury it a bit.
    offset=-2e6,
    fx=1.0,
    fy=2.0,
    source_time_function=simple_config.stf.Delta(),
)


w = simple_config.simulation.Waveform()
w.set_mesh(mesh)
w.add_receivers(recs)
w.add_sources(srcs)

w